# 1) Prodcue distance raster from point data

## 1.1 points inside 100m grid

In [5]:

## point to overlay with gridcell


import geopandas as gpd

# Load the polygon and point layers
polygons = gpd.read_file(r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\reference_units\GRID\grid31256\fishnet_100m_31256.shp")
points =   gpd.read_file(r'N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\sammelstellen\Altstoffsammelstellen_BIO_und_Mistplaetze_points_31256.shp')

output_grid_cell_with_points_inside = r'N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\Bio_sammel_mistplaetze_100m.shp'


# Perform spatial join to find polygons covered by points
selected_polygons = gpd.sjoin(polygons, points, how="inner", op="contains")

# Display or manipulate the selected polygons as needed
#print(selected_polygons.head())

# Save the selected polygons to a new shapefile if necessary
selected_polygons.to_file(output_grid_cell_with_points_inside , overwrite=True)



## add some new attriubtes to output-shp:

gdf = gpd.read_file(output_grid_cell_with_points_inside)
# Define the values for the new attribute
gdf['raster_att'] = 1

#new_values = ['1']  # List of values corresponding to each feature in the shapefile
# Add the new attribute
#gdf['raster_att'] = new_values
gdf.to_file(output_grid_cell_with_points_inside , overwrite=True)
print ("done")


C:\ProgramData\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3400: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
C:\Users\ManuelLöhnertz\AppData\Local\Temp\ipykernel_2504\3321701658.py:20: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  selected_polygons.to_file(output_grid_cell_with_points_inside , overwrite=True)


done


# 1.2 raster from the resulting grids

In [10]:
##

## raster to vector using vector attriubte (working perfectly):

import geopandas as gpd
import rasterio
from rasterio.features import rasterize
from rasterio.transform import from_bounds

# Load the polygon layer
polygons = gpd.read_file(r'N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\Bio_sammel_mistplaetze_100m.shp')

output_raster = r'N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\Bio_sammel_mistplaetze_100m.tif'


# Define the attribute to use for raster values
attribute_to_rasterize = 'raster_att'

# Define raster properties
#xmin, ymin, xmax, ymax = polygons.total_bounds

xmin=	-11200
ymin=	331000
xmax=	18800
ymax=	354000

projection ='epsg:31256'


pixel_size = 100  # Adjust as needed


width = int((xmax - xmin) / pixel_size)
height = int((ymax - ymin) / pixel_size)
transform = from_bounds(xmin, ymin, xmax, ymax, width, height)

# Create an empty array to hold rasterized values
raster_array = rasterize(
    [(geom, value) for geom, value in zip(polygons.geometry, polygons[attribute_to_rasterize])],
    out_shape=(height, width),
    transform=transform,
    fill=0,
    all_touched=False,  # or True
    dtype=rasterio.uint8
)

# Write the raster to a GeoTIFF file
with rasterio.open(output_raster, "w", driver="GTiff", 
                   height=raster_array.shape[0], 
                   width=raster_array.shape[1], 
                   count=1, 
                   dtype=raster_array.dtype, 
                   crs=polygons.crs, 
                   transform=transform
                   ) as dst:
    dst.write(raster_array, 1)


print ("done")

done


# 2) Calculation the distance of every raster cell to the nearest raster cell with value (1 = location of biotonne)

In [11]:
## calc. 
# The gdal_proximity.py script generates a raster proximity map indicating the distance from the center of each pixel 
# to the center of the nearest pixel identified as a target pixel. 
# Target pixels are those in the source raster for which the raster pixel value is in the set of target pixel values.
# https://gdal.org/programs/gdal_proximity.html

import subprocess

# Full path to gdal_proximity.py
gdal_proximity_path = r"C:\ProgramData\anaconda3\Scripts\gdal_proximity.py"

# Input and output filenames
input_raster =  r'N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\Bio_sammel_mistplaetze_100m2.tif'
output_raster =  r'N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\Bio_sammel_mistplaetze_100m_distance_raster2.tif'


# Command to run gdal_proximity.py
cmd = ["python", gdal_proximity_path, input_raster, output_raster]

# Run the command
subprocess.run(cmd)

print ("done")


done


# normalized values:

In [12]:
import numpy as np
import rasterio


import rasterio
from rasterio.transform import from_origin
from rasterio import Affine


input_distance_raster = r'N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\Bio_sammel_mistplaetze_100m_distance_raster2.tif'
# Define the output file path
output_file = r'N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\Bio_sammel_mistplaetze_100m_distance_raster_lin_NORM02.tif'
pixel_size = 100

# Open the GeoTIFF file
with rasterio.open(input_distance_raster) as src:
    # Read the raster values
    raster_values = src.read(1)  # Change the band index if necessary

    # Print the shape of the raster values array
    print("Shape of raster values array:", raster_values.shape)

    # Print the minimum and maximum values of the raster
    print("Minimum value:", raster_values.min())
    print("Maximum value:", raster_values.max())


    # Example raster values
    #raster_values = np.random.randint(0, 232343, size=(100, 100))  # Replace with your actual raster values

    # Min-max normalization
    min_val = np.min(raster_values.min())
    max_val = np.max( raster_values.max())
    normalized_values = 1-((raster_values - min_val) / (max_val - min_val) )  # tonne = 1 

    # Verify the range of normalized values (should be between 0 and 1)
    #min_normalized = np.min(normalized_values)
    #max_normalized = np.max(normalized_values)
    #print("Min normalized value:", min_normalized)
    #print("Max normalized value:", max_normalized)



    # Define the transformation parameters based on the dimensions of the raster and pixel size
    # Adjust these parameters according to your raster dimensions and pixel size
    transform = from_origin(0, 0, pixel_size, pixel_size)

    # Create a raster profile
    profile = src.profile

    # Write the normalized values to the raster file
    with rasterio.open(output_file, 'w', **profile) as dst:
        dst.write(normalized_values, 1)  # Write the data to the first band

print("Raster file written successfully:", output_file)

Shape of raster values array: (230, 300)
Minimum value: 0.0
Maximum value: 104.23531
Raster file written successfully: N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\Bio_sammel_mistplaetze_100m_distance_raster_lin_NORM02.tif


The follwowing image shows the final distance raster wiht normalized raster values from 0-1. The location of a biotonne hast the
raster value 1.  The pixel value decreases with increasing distance from the bins.

![distance](images/example_distance_raster.JPG)